# Exercises Neural Networks

In [75]:
import math                                                             # Mathematical functions
import pandas as pd                                                     # Data manipulation
import numpy as np                                                      # Scientific computing
from tensorflow import keras                                            # Deep Learning library
from tensorflow.keras import Model                                      # Model class
from tensorflow.keras.layers import Input, Dense, BatchNormalization    # Layers
from tensorflow.python.keras.utils.np_utils import to_categorical       # One-hot encoding
from tensorflow. keras.optimizers import Adam                           # Optimizer
from livelossplot import PlotLossesKeras                                # Live plot

from Resources.Functions import *                                       # Custom functions

import warnings                                                         # Disable warnings
warnings.filterwarnings("ignore")                                       # Ignoring future dependency warning.

## Theoretical questions

### Question 0: Demo ANN (Artificial Neural Network)

In [76]:
# This is one Artificial Neuron with 2 inputs (1,1), 2 weights (0.3, -0.1) and 1 bias (-0.7)
inputANN = pd.array([1,1])
weightsANN = pd.array([0.3, -0.1])
biasANN = -0.7

integratie = np.dot(inputANN, weightsANN) + biasANN
print(f"Output Integration: {integratie}")

activation = 1 / (1 + math.exp(-integratie))
print(f"Output Activation: {activation}")

Output Integration: -0.5
Output Activation: 0.3775406687981454


### Question 0: XOR ANN (Artificial Neural Network)

In [77]:
# Stap 1: Data
xor_data = pd.DataFrame({'P1': [0,1,0,1], 'P2': [0,0,1,1], 'Q' : [0,1,1,0]})
x_xor_data= xor_data[['P1', 'P2']]
y_xor_data = xor_data[['Q']]
                       
# Step 2: Model
inputs_xor = Input(shape=(2,))
x_xor = Dense(2, activation='sigmoid')(inputs_xor) # Or relu
outputs_xor = Dense(1, activation='sigmoid')(x_xor) # Or relu
model_xor = Model(inputs_xor, outputs_xor, name='XOR_NN')

# Step 3: Compile
model_xor.compile(optimizer=Adam(learning_rate=0.00001), loss=keras.losses.binary_crossentropy, metrics=['accuracy'])

# Step 4: Fit
history_xor = model_xor.fit(
                            x_xor_data, 
                            y_xor_data, 
                            epochs=200, 
                            callbacks=[PlotLossesKeras()], 
                            verbose=False)

accuracy
	accuracy         	 (min:    0.500, max:    0.500, cur:    0.500)
Loss
	loss             	 (min:    0.696, max:    0.697, cur:    0.696)


In [78]:
model_xor.summary()

Model: "XOR_NN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 2)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 2)              │             6 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 1)              │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29 (120.00 B)

 Trainable params: 9 (36.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 20 (84.00 B)

### Question 1:
- Bouw een neuraal netwerk op om het resultaat van een student op de gapminder test te bepalen aan de hand van het aantal uren wiskunde in het laatste jaar van het middelbaar, het aantal complotheorieën waarin hij/zij gelooft (zie code vraag 1 vragenlijst les Evaluatimetrieken) en het aantal broers en zussen. Splits de gegevens op is een training en een test data set (`90-10` verhouding). Normaliseer ook de data met behulp van de decimal scaling.

- Opmerking: de variabele gapminder test bevat niet getallen. 
    - Zet de data om naar een geheel getal –integer- dat het percentage weergeeft.
    - Plaats daarna de gegevens in `5` klassen: (`0,20`), (`20,40`), (`40,60`), (`60,80`), (`80,100`).
    - Vervang de klasse door de index van de klasse.

**Name of Columns**: `gapminder` `schrijfhand` `schoenmaat` `lengte` `bloedgroep` `resus` `siblings` `Samenzweringen` `rijbewijs` `domicilie in Antwerpen` `afstudeerrichting` `vrijetijdsbesteding` `uren wiskunde` `informatica` `belangrijk` `uren buiten schooluren` `afstand tot KdG` `vervoersmiddel` `klimaatopwarming` `zakgeld` `stukken fruit` `fruitvoorkeur` `besturingssysteem` `belangrijkste app` `M1 - stelen` `M2 - team` `M3 - beschermen` `M4 - voorbeeld` `M5 - embryo` `M6 - vrijheid` `mascotte` `zwaarste vak` `boeienste vak` `studiepunt`

In [79]:
data1 = pd.read_csv('../Data/Data1.csv', delimiter=';', decimal=',')
display(data1.head())

,gapminder,schrijfhand,schoenmaat,lengte,bloedgroep,resus,siblings,Samenzweringen,rijbewijs,domicilie in Antwerpen,...,M1 - stelen,M2 - team,M3 - beschermen,M4 - voorbeeld,M5 - embryo,M6 - vrijheid,mascotte,zwaarste vak,boeienste vak,studiepunt
0,7/18,Rechts,43.0,184,O,positief,5,De tsunami van 2004 in de Indische Oceaan was ...,Categorie B: Wagens.,"buiten de stad (een dorp, een gehucht, het pla...",...,7,2,8,6,5,4.0,"1=Bever,2=Wasbeer,3=Otter,4=Wolf,5=Poema,6=Pan...",Management 1,Data science 1,16-20 uren
1,3/18,Rechts,42.0,163,Weet ik niet,Weet ik niet,1,Prinses Diana kwam niet per ongeluk om het leven,Categorie AM: Bromfietsen met een maximumsnelh...,in een stad,...,10,5,10,6,7,3.0,"1=Bever,2=Wolf,3=Bizon,4=Otter,5=Panter,6=Poem...",Management 1,Computersystemen 1,<= 10 uren
2,12/18,Rechts,45.0,183,AB,positief,2,Prinses Diana kwam niet per ongeluk om het lev...,Categorie AM: Bromfietsen met een maximumsnelh...,in een stad,...,6,3,7,7,10,6.0,"1=Bruine beer,2=Poema,3=Panter,4=Wolf,5=Vos,6=...",Management 1,Programmeren 1 - Java,16-20 uren
3,5/18,Rechts,44.0,175,A,positief,2,De wereld wordt eigenlijk gecontroleerd door e...,Categorie AM: Bromfietsen met een maximumsnelh...,in een stad,...,9,10,10,10,10,0.0,"1=Wasbeer,2=Wolf,3=Bruine beer,4=Das,5=Vos,6=B...",Management 1,Data science 1,<= 10 uren
4,6/18,Rechts,43.0,173,B,positief,3,De wereld wordt eigenlijk gecontroleerd door e...,Categorie B: Wagens.,in een stad,...,10,6,10,7,7,2.0,"1=Wolf,2=Vos,3=Bruine beer,4=Bizon,5=Panter,6=...",Management 1,Data science 1,<= 10 uren


### Question 2:
- Herneem het neuraal netwerk uit vraag 1. Maak de confusion matrix, bereken de meest gebruikte evaluatiemetrieken en teken de ROC.

### Question 3: The Simpsons Revisited
- We willen het geslacht van een Simpson kunnen voorspellen m.b.v. een neuraal netwerk. Deze dataset is erg klein en daarom kunnen we geen validatie-set of test-set construeren. 
    - Lees de dataset (`The_Simpsons_Original.csv`) van de Simpsons in. 
    - Bepaal welke kolommen we niet kunnen gebruiken bij deze voorspelling?
    - Maak nu een input training data set `x_simpsons` en normaliseer deze (min-max normalisatie). Doe hetzelfde voor de outputkolom(y4_simpsons).
    - Maak een neuraal netwerk met de correcte inputs en outputs.
    - Gebruik je neuraal netwerk om het geslacht van je `x_simpsons` te voorspellen. Komen ze overeen met de targets? Voorspel ook het geslacht van de onbekende Simpson (Comic guy: `hairlenght=8`, `weight=500`, `age=38`).. Komt dit overeen met de voorspelling die je gemaakt werd bij beslissingsbomen?

**Name of Columns**: `name` `hair length` `weight` `age` `gender`

In [80]:
# 1
theSimpsonsOriginal = pd.read_csv('../Data/The_Simpsons_Original.csv', delimiter=',')
display(theSimpsonsOriginal.head())

,name,hair length,weight,age,gender
0,Homer,0,250,36,M
1,Marge,10,150,34,F
2,Bart,2,90,10,M
3,Lisa,6,78,8,F
4,Maggie,4,20,1,F


In [81]:
# 2
theSimpsonsOriginal = theSimpsonsOriginal.drop(columns=['name'])
display(theSimpsonsOriginal.head())

,hair length,weight,age,gender
0,0,250,36,M
1,10,150,34,F
2,2,90,10,M
3,6,78,8,F
4,4,20,1,F


In [104]:
# 3
x_simpsons = theSimpsonsOriginal[['hair length', 'weight', 'age']]                                              # Get the input
y_simpsons = theSimpsonsOriginal[['gender']]                                                                    # Get the output

x_simpsons_norm = (x_simpsons - x_simpsons.min()) / (x_simpsons.max() - x_simpsons.min())                       # Normalization of the input
y_simpsons_rep = y_simpsons.replace(to_replace=['M', 'F'], value=[0, 1])                                        # Replace the output
y_simpsons_rep = to_categorical(y_simpsons_rep)                                                                 # One-hot encoding
y_simpsons_norm = (y_simpsons_rep - y_simpsons_rep.min()) / (y_simpsons_rep.max() - y_simpsons_rep.min())       # Normalization of the output

In [105]:
# 4
inputs_simpsons = Input(shape=(3,))
x_simp = Dense(6, activation='sigmoid')(inputs_simpsons)
x_simp = Dense(12, activation='sigmoid')(x_simp)
x_simp = Dense(24, activation='sigmoid')(x_simp)
x_simp = Dense(12, activation='sigmoid')(x_simp)
x_simp = Dense(6, activation='sigmoid')(x_simp)
outputs_simpsons = Dense(2, activation='softmax')(x_simp)
model_simpsons = Model(inputs_simpsons, outputs_simpsons, name='Simpsons_NN')

model_simpsons.compile(optimizer=Adam(learning_rate=0.00001), loss=keras.losses.binary_crossentropy, metrics=['accuracy'])

history_simpsons = model_simpsons.fit(
                                    x_simpsons_norm,
                                    y_simpsons_rep,
                                    epochs=200,
                                    callbacks=[PlotLossesKeras()],
                                    verbose=False)

model_simpsons.summary()

accuracy
	accuracy         	 (min:    0.556, max:    0.556, cur:    0.556)
Loss
	loss             	 (min:    0.709, max:    0.711, cur:    0.709)


Model: "Simpsons_NN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_25 (InputLayer)     │ (None, 3)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_130 (Dense)               │ (None, 6)              │            24 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_131 (Dense)               │ (None, 12)             │            84 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_132 (Dense)               │ (None, 24)             │           312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_133 (Dense)               │ (None, 12)             │           300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_134 (Dense)               │ (None, 6)              │            78 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_135 (Dense)               │ (None, 2)              │            14 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,438 (9.53 KB)

 Trainable params: 812 (3.17 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,626 (6.36 KB)

In [107]:
# 5
comicGuy = pd.DataFrame({'hair length': [8], 'weight': [500], 'age': [38]})
comicGuy_norm = (comicGuy - x_simpsons.min()) / (x_simpsons.max() - x_simpsons.min())
prediction = model_simpsons.predict(comicGuy_norm)
print(f"Prediction: {prediction}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Prediction: [[0.62706274 0.37293726]]


### Question 4: Forecast demo Revisited
- We willen de opbrengsten van een bedrijf per kwartaal van de laatste 5 jaar kunnen voorspellen. Deze dataset is erg klein en daarom kunnen we geen validatie-set of test-set construeren. 
    - Lees de dataset van de Forecasting in (`forecastdemo.csv`)
    - Wat voor soort voorspelling trachten we hier te doen? Regressie of classificatie?
    - Bepaal welke kolommen we niet kunnen gebruiken bij deze voorspelling?
    - Maak nu een input training data set x_forcast en een outputkolom (`y_forcast`). Normaliseer waar nodig.
    - Maak een neuraal netwerk met de correcte inputs en outputs. Maak een plot van de ANN.
    - Gebruik je neuraal netwerk om de opbrengsten te voorspellen die horen bij `x_forcast`. Komen ze overeen met de werkelijke opbrengsten? Maak één plot van de originele gegevens en de voorspelde gegevens.

In [ ]:
# 1
forcastDemo = pd.read_csv('../Data/Forcastdemo.csv', delimiter=';')
display(forcastDemo.head())

In [ ]:
# 2

### Question 5: Iris
- Laad de dataset iris (`iris.csv`) op en bestudeer de inhoud.
- Bepaal welke kolommen we niet kunnen gebruiken bij deze voorspelling?
- Maak nu een input data set `x_iris` en normaliseer deze. Probeer een van de normalisatietechnieken uit die je nog niet hebt gebruikt in een oefening of opdracht.
- Creëer een trainig data set en een test data set (`x_train_iris` en `x_test_iris`) die respectievelijk bestaan uit `80%` en `20%` (willekeurig gekozen) van de input data set `x_iris`.
- Maak een neuraal netwerk met de correcte inputs en outputs.
- Evalueer je neuraal netwerk op je test data set (`x_test_iris`). Welke nauwkeurigheid (accuracy) behaal je?

**Name of Columns**: `sepal length` `sepal width` `petal length` `petal width` `target`

In [108]:
# 1
iris = pd.read_csv('../Data/Iris.csv', delimiter=',', decimal='.')
display(iris.head())

,sepal length,sepal width,petal length,petal width,target
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [ ]:
# 2